In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

In [10]:
#import sheets
file_path = '../data/E-commerce_data.xlsx'

customers = pd.read_excel(file_path, sheet_name='customers')
genders = pd.read_excel(file_path, sheet_name='genders')
cities = pd.read_excel(file_path, sheet_name='cities')
transactions = pd.read_excel(file_path, sheet_name='transactions')
branches = pd.read_excel(file_path, sheet_name='branches')
merchants = pd.read_excel(file_path, sheet_name='merchants')


In [ ]:
# Merge all datasets
data = pd.merge(transactions, customers, how='left', on='customer_id')
data = pd.merge(data, genders, how='left', on='gender_id')
data = pd.merge(data, cities, how='left', on='city_id')
data = pd.merge(data, branches, how='left', on='branch_id')
data = pd.merge(data, merchants, how='left', on='merchant_id')

In [ ]:
print("Dataset shape:", data.shape)
print("\nFirst few rows:")
print(data.head())

In [ ]:
# Data preprocessing
data['transaction_date'] = pd.to_datetime(data['transaction_date'])
data['burn_date'] = pd.to_datetime(data['burn_date'], errors='coerce')
data['join_date'] = pd.to_datetime(data['join_date'], errors='coerce')

# Feature Selection

In [ ]:
def create_customer_features(df):
    customer_stats = df.groupby('customer_id').agg({
        'transaction_date': ['max', 'count', lambda x: (pd.Timestamp.now() - x.max()).days],  # Recency, Frequency
        'transaction_id': 'nunique',  # Unique transactions
        'transaction_status': lambda x: (x == 'burned').sum(),  # Coupon burns
        'city_name': lambda x: x.nunique(),  # City diversity
        'merchant_name': lambda x: x.nunique(),  # Merchant diversity
        'branch_id': lambda x: x.nunique(),  # Branch diversity
        'gender_name': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown'
    }).round(2)

    customer_stats.columns = ['LastPurchase', 'Frequency', 'Recency', 'UniqueTransactions',
                            'CouponUsage', 'CityDiversity', 'MerchantDiversity',
                            'BranchDiversity', 'PrimaryGender']

    # Add Monetary value (proxy: transaction count weighted by recency)
    customer_stats['Monetary'] = customer_stats['Frequency'] * (1 / (1 + customer_stats['Recency']/30))

    return customer_stats.reset_index()

customer_data = create_customer_features(data)
print("\nCustomer features shape:", customer_data.shape)
print(customer_data.head())

# Prepare features for clustering
numerical_features = ['Recency', 'Frequency', 'Monetary', 'UniqueTransactions',
                     'CouponUsage', 'CityDiversity', 'MerchantDiversity', 'BranchDiversity']

X = customer_data[numerical_features].fillna(0)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\nScaled features shape: {X_scaled.shape}")

# Distribution of Features

In [ ]:
# Convert the scaled data back to DataFrame for better visualization
X_scaled_df = pd.DataFrame(X_scaled, columns=numerical_features)

# Distribution of Features (Histograms)
plt.figure(figsize=(14, 8))
X_scaled_df.hist(bins=20, edgecolor='black', figsize=(14, 8))
plt.suptitle('Distribution of Customer Features', fontsize=16)
plt.tight_layout()
plt.show()

# Correlation Heatmap

In [ ]:
plt.figure(figsize=(10, 6))
correlation_matrix = X_scaled_df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', fmt='.2f', linewidths=0.5)
plt.title('Correlation Heatmap of Customer Features', fontsize=16)
plt.show()

# Pair Plot

In [ ]:
sns.pairplot(X_scaled_df)
plt.suptitle('Pair Plot of Customer Features', fontsize=16)
plt.show()

# Box Plots for each feature to identify outliers

In [ ]:
plt.figure(figsize=(14, 8))
for i, feature in enumerate(numerical_features, 1):
    plt.subplot(2, 4, i)
    sns.boxplot(data=X_scaled_df, x=feature)
    plt.title(f'Boxplot of {feature}')

plt.tight_layout()
plt.show()

# K-Means Clustering

In [ ]:
inertias = []
silhouette_scores = []

for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, labels))

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(range(2, 11), inertias, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')

plt.subplot(1, 2, 2)
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.title('Silhouette Score')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.tight_layout()
plt.show()

def find_optimal_k(inertias, silhouette_scores):
    """Better method combining Elbow + Silhouette"""
    # Elbow method: Look for biggest drop in inertia
    inertia_diffs = np.diff(inertias)
    elbow_k = np.argmax(inertia_diffs) + 2

    # Silhouette peak
    sil_peak_k = np.argmax(silhouette_scores) + 2

    # Business preference: 3-6 clusters for marketing
    print(f"Elbow suggests: k={elbow_k}")
    print(f"Silhouette peak: k={sil_peak_k}")
    print(f"Recommended: k=4 (business optimal)")

    return 4  # Force optimal business value

optimal_k = find_optimal_k(inertias, silhouette_scores)

# Visualization with PCA

In [ ]:
# Apply PCA to reduce to 3 components
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_scaled)

# KMeans model
kmeans = KMeans(init='k-means++', n_clusters=optimal_k, random_state=42, n_init=10)
customer_data['KMeans_Cluster'] = kmeans.fit_predict(X_scaled)
print(f"K-Means Optimal Clusters: {optimal_k}")

# Create a single plot for KMeans
fig, ax = plt.subplots(figsize=(10, 8))

# Scatter plot: color by the clustering result
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=customer_data['KMeans_Cluster'], cmap='viridis', s=50)

# Plot centroids: Use PCA to transform KMeans centroids to the 2D space
centroids_pca = pca.transform(kmeans.cluster_centers_)

# Plot centroids on the scatter plot
ax.scatter(centroids_pca[:, 0], centroids_pca[:, 1], c='red', marker='X', s=200, label='Centroids')

# Set title and axis labels
ax.set_title('KMeans Clustering (PCA)', fontsize=16)
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')

# Add colorbar to the plot
plt.colorbar(scatter, ax=ax)

# Add legend for centroids
ax.legend()

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

# Cluster Profiling

In [ ]:
# Cluster Profiling
def profile_clusters(customer_data, cluster_col):
    print(f"\n=== {cluster_col} Profiles ===")
    profile = customer_data.groupby(cluster_col)[numerical_features].mean().round(2)
    print(profile)

    plt.figure(figsize=(15, 10))
    for i, feature in enumerate(numerical_features):
        plt.subplot(3, 3, i+1)
        sns.boxplot(data=customer_data, x=cluster_col, y=feature)
        plt.title(feature)
        plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

profile_clusters(customer_data=customer_data, cluster_col='KMeans_Cluster')

# Marketing Insights

In [ ]:
print("\n=== MARKETING INSIGHTS ===")

# 1. VIP Customers (Top 20% Frequency + Monetary)
print("1. VIP Customers (Top 20% Frequency + Monetary):")
vip_customers = customer_data[
    (customer_data['Frequency'] > customer_data['Frequency'].quantile(0.8)) &
    (customer_data['Monetary'] > customer_data['Monetary'].quantile(0.8))
]
print(f"Count: {len(vip_customers)} ({len(vip_customers)/len(customer_data)*100:.1f}%)")
print(f"Avg Frequency: {vip_customers['Frequency'].mean():.1f}")
print("Target these customers with exclusive offers and loyalty programs!")

# 2. Coupon Enthusiasts
print("\n2. Coupon Enthusiasts:")
coupon_users = customer_data[
    customer_data['CouponUsage'] > customer_data['CouponUsage'].quantile(0.75)
]
print(f"Count: {len(coupon_users)}")
print("Target with coupon campaigns and personalized offers!")

# 3. At-Risk Customers (Inactive > 90 days)
print("\n3. At-Risk Customers (Inactive > 90 days):")
risky = customer_data[customer_data['Recency'] > 90]
print(f"Count: {len(risky)} - Re-engagement campaigns needed!")

# 4. Cluster-Specific Insights Based on K-Means Results
print("\n4. Cluster-Specific Insights:")

# Cluster-wise breakdown:
for cluster_num in customer_data['KMeans_Cluster'].unique():
    cluster_data = customer_data[customer_data['KMeans_Cluster'] == cluster_num]
    print(f"\nCluster {cluster_num}:")
    print(f" - Count: {len(cluster_data)} customers")
    print(f" - Avg Frequency: {cluster_data['Frequency'].mean():.1f}")
    print(f" - Avg Recency: {cluster_data['Recency'].mean():.1f} days")
    print(f" - Avg Monetary: {cluster_data['Monetary'].mean():.1f}")

    # Custom insights based on cluster characteristics
    if cluster_num == 0:  # Example: High Frequency, High Recency, Low Monetary
        print("  * These customers are very engaged but spend less. Focus on upselling or cross-selling higher-value items.")
        print("  * Target with loyalty rewards and personalized offers to increase average spend per purchase.")

    elif cluster_num == 1:  # Example: Low Frequency, High Recency, High Monetary
        print("  * These customers spend a lot but don't shop often. Create exclusive, time-limited offers to encourage repeat purchases.")
        print("  * Focus on high-value, personalized email campaigns and VIP rewards.")

    elif cluster_num == 2:  # Example: Low Recency, Low Frequency, Low Monetary
        print("  * These customers are at risk of churn. Target them with win-back campaigns, offering discounts or personalized recommendations.")
        print("  * Implement re-engagement strategies to rekindle interest.")

    elif cluster_num == 3:  # Example: High Frequency, Low Recency, High Monetary
        print("  * These customers were high spenders but have become inactive. Reach out with exclusive re-engagement offers to bring them back.")
        print("  * Offer loyalty bonuses and show them relevant product recommendations based on their past purchases.")

print("\n=== End of Marketing Insights ===")


# Save results

In [ ]:
output_dir = '../outputs/results'
os.makedirs(output_dir, exist_ok=True)

customer_data.to_csv(os.path.join(output_dir, 'customer_segments_complete.csv'), index=False)
print("\nResults saved to 'customer_segments_complete.csv'")
print("\nClustering Summary:")
print(f"• K-Means: {optimal_k} clusters")